In [1]:
import pandas as pd
import numpy as np
import logging
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.datasets import load_iris
from datetime import datetime

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_data():
    """Cargar el dataset y devolverlo como DataFrame de pandas"""
    logger.info("Cargando el dataset...")
    data = load_iris(as_frame=True)
    df = data.frame
    logger.info("Dataset cargado con éxito")
    return df

def preprocess_data(df):
    """Limpiar y preparar datos para entrenamiento"""
    logger.info("Iniciando preprocesamiento de datos...")
    X = df.drop(columns=['target'])
    y = df['target']
    
    # Identificar tipos de columnas
    categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
    numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
    
    # Pipeline de preprocesamiento
    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ])
    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Composición final del preprocesador
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_cols),
            ('cat', categorical_pipeline, categorical_cols)
        ])
    
    logger.info("Preprocesamiento completo.")
    return preprocessor, X, y

def train_model(preprocessor, X, y):
    """Entrenar el modelo de clasificación - en árbol de decisiones"""
    logger.info("Dividiendo los datos en conjunto de entrenamiento y prueba...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Pipeline final de entrenamiento
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ])
    
    # Entrenar el modelo
    logger.info("Entrenando el modelo...")
    pipeline.fit(X_train, y_train)
    logger.info("Entrenamiento completado.")
    
    # Evaluación del modelo
    logger.info("Evaluando el modelo...")
    y_pred = pipeline.predict(X_test)
    logger.info("\n" + classification_report(y_test, y_pred))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    logger.info(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')
    
    return pipeline

def save_model(model, filename):
    """Guardar el modelo entrenado en un archivo .pkl"""
    logger.info(f"Guardando el modelo en {filename}...")
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    logger.info("Modelo guardado con éxito.")

def main():
    logger.info("Nombre: Gabriel Guzman")
    
    logger.info("Inicio del pipeline de clasificación.")
    df = load_data()
    preprocessor, X, y = preprocess_data(df)
    model = train_model(preprocessor, X, y)
    save_model(model, f'model_{datetime.now().strftime("%Y%m%d%H%M")}.pkl')
    logger.info("Pipeline completado con éxito.")

if __name__ == "__main__":
    main()


2024-11-01 19:54:17,580 - INFO - Nombre: Gabriel Guzman
2024-11-01 19:54:17,581 - INFO - Inicio del pipeline de clasificación.
2024-11-01 19:54:17,582 - INFO - Cargando el dataset...
2024-11-01 19:54:17,614 - INFO - Dataset cargado con éxito
2024-11-01 19:54:17,615 - INFO - Iniciando preprocesamiento de datos...
2024-11-01 19:54:17,618 - INFO - Preprocesamiento completo.
2024-11-01 19:54:17,619 - INFO - Dividiendo los datos en conjunto de entrenamiento y prueba...
2024-11-01 19:54:17,622 - INFO - Entrenando el modelo...
2024-11-01 19:54:17,631 - INFO - Entrenamiento completado.
2024-11-01 19:54:17,632 - INFO - Evaluando el modelo...
2024-11-01 19:54:17,642 - INFO - 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30